In [ ]:
#Musical Tracks Many-to-One  

#Create Tables
CREATE TABLE album (
  id SERIAL,
  title VARCHAR(128) UNIQUE,
  PRIMARY KEY(id)
);

CREATE TABLE track (
    id SERIAL,
    title VARCHAR(128),
    len INTEGER, rating INTEGER, count INTEGER,
    album_id INTEGER REFERENCES album(id) ON DELETE CASCADE,
    UNIQUE(title, album_id),
    PRIMARY KEY(id)
);

DROP TABLE IF EXISTS track_raw;
CREATE TABLE track_raw
    (title TEXT, artist TEXT, album TEXT, album_id INTEGER,
    count INTEGER, rating INTEGER, len INTEGER);
    
#Load this CSV data file into the track_raw table using the \copy command
\copy track_raw(title,artist,album,count,rating,len) FROM 'library.csv' WITH DELIMITER ',' CSV;       

#Then write SQL commands to insert all of the distinct albums into the album table (creating their primary keys) 
INSERT INTO album (title) SELECT DISTINCT album FROM track_raw;

#set the album_id in the track_raw table using an SQL query like: 
UPDATE track_raw SET album_id = (SELECT album.id FROM album WHERE album.title = track_raw.album);

#Then use a INSERT ... SELECT statement to copy the corresponding data from the track_raw table to the track table, effectively dropping the artist and album text fields. 
INSERT INTO track (title, len, rating, count, album_id) SELECT title, len, rating, count, album_id FROM track_raw;

#Run Query
SELECT track.title, album.title
    FROM track
    JOIN album ON track.album_id = album.id
    ORDER BY track.title LIMIT 3;

In [ ]:
#Unesco Heritage Sites Many-to-One

#Create unesco_raw table
DROP TABLE unesco_raw;
CREATE TABLE unesco_raw (
  name TEXT,
  description TEXT,
  justification TEXT,
  year INTEGER,
  longitude FLOAT,
  latitude FLOAT,
  area_hectares FLOAT,
  category TEXT,
  category_id INTEGER,
  state TEXT,
  state_id INTEGER,
  region TEXT,
  region_id INTEGER,
  iso TEXT,
  iso_id INTEGER
);

#Create 4 tables, category, state, region, iso
CREATE TABLE category (
  id SERIAL,
  name VARCHAR(128) UNIQUE,
  PRIMARY KEY(id)
);

CREATE TABLE state (
  id SERIAL,
  name VARCHAR(128) UNIQUE,
  PRIMARY KEY(id)
);

CREATE TABLE region (
  id SERIAL,
  name VARCHAR(128) UNIQUE,
  PRIMARY KEY(id)
);

CREATE TABLE iso (
  id SERIAL,
  name VARCHAR(128) UNIQUE,
  PRIMARY KEY(id)
);

#Copy csv file into unesco_raw
\ copy unesco_raw(
    name,
    description,
    justification,
    year,
    longitude,
    latitude,
    area_hectares,
    category,
    state,
    region,
    iso
)FROM'whc-sites-2018-small.csv' WITH DELIMITER ',' CSV HEADER;

#Insert data from unesco_raw to the 4 tables
INSERT INTO category(name) SELECT DISTINCT category FROM unesco_raw;

INSERT INTO state(name) SELECT DISTINCT state FROM unesco_raw;

INSERT INTO region(name) SELECT DISTINCT region FROM unesco_raw;

INSERT INTO iso(name) SELECT DISTINCT iso FROM unesco_raw;

#Hook up the 4 foreign keys with unesco_raw
UPDATE unesco_raw SET category_id = (
SELECT category.id FROM category WHERE category.name = unesco_raw.category);

UPDATE unesco_raw SET state_id = (SELECT state.id FROM state WHERE state.name = unesco_raw.state);

UPDATE unesco_raw SET region_id = (SELECT region.id FROM region WHERE region.name = unesco_raw.region);

UPDATE unesco_raw SET iso_id = (SELECT iso.id FROM iso WHERE iso.name = unesco_raw.iso);

#Create table unesco
CREATE TABLE unesco (
    name TEXT,
    description TEXT,
    justification TEXT,
    year INTEGER,
    longitude FLOAT,
    latitude FLOAT,
    area_hectares FLOAT,
    category_id INTEGER,
    state_id INTEGER,
    region_id INTEGER,
    iso_id INTEGER
);

#Insert data from unesco_raw into unesco
INSERT INTO unesco(
    name,
    description,
    justification,
    year,
    longitude,
    latitude,
    area_hectares,
    category_id,
    state_id,
    region_id,
    iso_id
  ) SELECT
  name,
  description,
  justification,
  year,
  longitude,
  latitude,
  area_hectares,
  category_id,
  state_id,
  region_id,
  iso_id FROM unesco_raw;

#Run Query
SELECT unesco.name, year, category.name, state.name, region.name, iso.name
  FROM unesco
  JOIN category ON unesco.category_id = category.id
  JOIN iso ON unesco.iso_id = iso.id
  JOIN state ON unesco.state_id = state.id
  JOIN region ON unesco.region_id = region.id
  ORDER BY state.name, unesco.name
  LIMIT 3;

In [ ]:
#Musical Track Database plus Artists

#Create 4 tables
DROP TABLE album CASCADE;
CREATE TABLE album (
    id SERIAL,
    title VARCHAR(128) UNIQUE,
    PRIMARY KEY(id)
);

DROP TABLE track CASCADE;
CREATE TABLE track (
    id SERIAL,
    title TEXT, 
    artist TEXT, 
    album TEXT, 
    album_id INTEGER REFERENCES album(id) ON DELETE CASCADE,
    count INTEGER, 
    rating INTEGER, 
    len INTEGER,
    PRIMARY KEY(id)
);

DROP TABLE artist CASCADE;
CREATE TABLE artist (
    id SERIAL,
    name VARCHAR(128) UNIQUE,
    PRIMARY KEY(id)
);

DROP TABLE tracktoartist CASCADE;
CREATE TABLE tracktoartist (
    id SERIAL,
    track VARCHAR(128),
    track_id INTEGER REFERENCES track(id) ON DELETE CASCADE,
    artist VARCHAR(128),
    artist_id INTEGER REFERENCES artist(id) ON DELETE CASCADE,
    PRIMARY KEY(id)
);

#Copy data from csv file to table called track
\copy track(title,artist,album,count,rating,len) FROM 'library.csv' WITH DELIMITER ',' CSV;

INSERT INTO album (title) SELECT DISTINCT album FROM track;

UPDATE track SET album_id = (SELECT album.id FROM album WHERE album.title = track.album);

INSERT INTO tracktoartist (track, artist) SELECT DISTINCT title, artist FROM track;
INSERT INTO artist (name) SELECT DISTINCT artist FROM track;

UPDATE tracktoartist SET track_id = (SELECT track.id FROM track WHERE track.title = tracktoartist.track);
UPDATE tracktoartist SET artist_id = (SELECT artist.id FROM artist WHERE artist.name = tracktoartist.artist);

#Alter and drop the columns as required
ALTER TABLE track DROP COLUMN album;
ALTER TABLE track DROP COLUMN artist;
ALTER TABLE tracktoartist DROP COLUMN track;
ALTER TABLE tracktoartist DROP COLUMN artist;

#Run Query
SELECT track.title, album.title, artist.name
FROM track
JOIN album ON track.album_id = album.id
JOIN tracktoartist ON track.id = tracktoartist.track_id
JOIN artist ON tracktoartist.artist_id = artist.id
LIMIT 3;